In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# image 3 * 224 * 224 assumed
class VGG(nn.Module): # use this for CIFAR10 datasets
    def __init__(self, layer_list, num_class=10):
        super().__init__()
        self.vgg_layer_list = []
        self.initial_channel = 3
        for m in layer_list:
            if m == 'M':
                self.vgg_layer_list.append(nn.MaxPool2d(2))
            else:
                self.vgg_layer_list += [
                    nn.Conv2d(self.initial_channel, m, kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(m),
                    nn.ReLU(inplace=True)]
                self.initial_channel = m
                
        self.convnet = nn.Sequential(*self.vgg_layer_list)
        self.avgpool = nn.AdaptiveAvgPool2d(7) # 뭘까?????
        self.FClayer = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_class) # softmax skip
        )
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                else:
                    pass # 수정
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant(m.weight, 1)
                nn.init.constant(m.bias, 0)
            else:
                pass
    
    def forward(self, x):
        out = self.avgpool(self.convnet(x))
        out = out.view(out.shape[0], -1)
        out = self.FClayer(out)
        return out
                
layer_list = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], #8 + 3 =11 == vgg11
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], # 10 + 3 = vgg 13
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'], #13 + 3 = vgg 16
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'], # 16 +3 =vgg 19
    'custom' : [64,64,64,'M',128,128,128,'M',256,256,256,'M']
}

VGGnet = VGG(layer_list['D'], 10).to(device)
VGGnet

In [10]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = torchvision.datasets.CIFAR10(root='.CIFAR10', train=True,
                                              transform=trans, download=True)
testset = torchvision.datasets.CIFAR10(root='.CIFAR10', train=False,
                                              transform=trans, download=True)

data_loader_train = DataLoader(trainset, batch_size=512,
                               shuffle=True, drop_last=True)
data_loader_test = DataLoader(testset, batch_size=32,
                               shuffle=False, drop_last=True)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(VGGnet.parameters(), lr=0.001, momentum=0.9)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9) # learning rate를 변경해주는 scheduler

In [11]:
x = torch.randn(1,3,224,224).to(device)
VGGnet(x)

In [12]:
def test_model(model, data_loader_test):
    total = 0
    correct = 0
    with torch.no_grad():
        model.eval()
        for data in data_loader_test:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            out = model(images)
            total += labels.size(0)
            correct += (torch.argmax(out, 1) == labels).sum().item()
        print('Accuracy : {}'.format(100 * correct / total))

In [13]:
def train_model(model, data_loader_train, data_loader_test, optimizer, criterion, lr_schedule=None):
    model.train()
    for epoch in range(30):
        running_loss = 0.0
        lr_sche.step() # update learning rate
        
        for i, data in enumerate(data_loader_train):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()

            hypothesis = model(inputs)
            loss = criterion(hypothesis, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() / 512
        print("Epoch : %d, Loss : %f"%(epoch + 1, running_loss))
        running_loss = 0
        test_model(model, data_loader_test)

In [14]:
train_model(VGGnet, data_loader_train, data_loader_test, optimizer, criterion, lr_sche)